In [20]:
import pandas as pd
import numpy as np
from glob import glob
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# csv 확장자인 모든 파일 경로 읽기
files = glob('*.csv')

dfs = []
for file in files:
    enc = detect_encoding(file)
    try:
        df = pd.read_csv(file, encoding=enc)
    except Exception as e:
        print(f"Error reading {file} with encoding {enc}: {e}")
        df = pd.read_csv(file, encoding='utf-8', errors='ignore')
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)


In [21]:
df = merged_df[['종류','유형','시설명','주소','위도','경도','정원']]

In [71]:
df.head()

,종류,유형,시설명,주소,위도,경도,정원
0,노인여가복지시설,노인복지관,강화군노인복지관,인천광역시 강화군 강화읍 중앙로 17-16,37.740321,126.491208,7176
1,노인여가복지시설,노인복지관,강화군노인문화센터,인천광역시 강화군 길상면 마니산로 36,37.642540,126.487185,1246
2,노인여가복지시설,경로당,신문리(대안) 경로당,인천광역시 강화군 강화읍 강화대로419번길 5-1,37.746450,126.484878,95
3,노인여가복지시설,경로당,"신문1,3,5리 경로당",인천광역시 강화군 강화읍 청하동길9번길 8,37.746676,126.482583,28
4,노인여가복지시설,경로당,"신문1,2,5리(부녀) 경로당",인천광역시 강화군 강화읍 청하동길9번길 8,37.746676,126.482583,37


In [72]:
df.isna().sum()

종류      579
유형     1241
시설명     173
주소      311
위도        0
경도        0
정원      914
dtype: int64

In [22]:
df = df.dropna()

In [23]:
df

,종류,유형,시설명,주소,위도,경도,정원
0,노인여가복지시설,노인복지관,강화군노인복지관,인천광역시 강화군 강화읍 중앙로 17-16,37.740321,126.491208,7176
1,노인여가복지시설,노인복지관,강화군노인문화센터,인천광역시 강화군 길상면 마니산로 36,37.642540,126.487185,1246
2,노인여가복지시설,경로당,신문리(대안) 경로당,인천광역시 강화군 강화읍 강화대로419번길 5-1,37.746450,126.484878,95
3,노인여가복지시설,경로당,"신문1,3,5리 경로당",인천광역시 강화군 강화읍 청하동길9번길 8,37.746676,126.482583,28
4,노인여가복지시설,경로당,"신문1,2,5리(부녀) 경로당",인천광역시 강화군 강화읍 청하동길9번길 8,37.746676,126.482583,37
...,...,...,...,...,...,...,...
1295,재가노인복지시설,재가노인복지시설,소리재가복지센터,인천광역시 서구 원당대로685번길 5,37.594363,126.677189,0.0
1296,재가노인복지시설,재가노인복지시설,코리아 의료기 복지용구,인천광역시 서구 건지로249번길 32-4,37.502034,126.670202,0.0
1297,재가노인복지시설,재가노인복지시설,최우수노인복지센터,인천광역시 서구 가정로 451,37.521956,126.676508,0.0
1298,재가노인복지시설,재가노인복지시설,다은 주간보호센터,인천광역시 서구 원당대로835번길 7,37.594700,126.694355,46.0


In [52]:
import folium

# 인천 중심 좌표 설정
incheon_center = [37.456255, 126.705206]

# 기본 지도 생성
m = folium.Map(location=incheon_center, zoom_start=11)

# 시설 위치 반복문 돌면서 지도에 마커 추가
for idx, row in df.iterrows():
    folium.Marker(location=[row['위도'], row['경도']],
                  popup=row['시설명']).add_to(m)

# 지도 출력 (주피터 노트북에서)
m


In [56]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# 시설 위도, 경도 배열
locations = df[['위도', '경도']].to_numpy()

# 위도, 경도를 라디안으로 변환
locations_rad = np.radians(locations)

# k-NN 모델 초기화 (3개 최근접 시설 찾기)
nbrs = NearestNeighbors(n_neighbors=3, metric='haversine')
nbrs.fit(locations_rad)

# 사용자 위치 (예: 위도, 경도)
user_location = (37.542996, 126.677156)  # 학원 위치
user_loc_rad = np.radians([user_location])

# 최근접 3개 시설 찾기
distances, indices = nbrs.kneighbors(user_loc_rad)

# 거리(km)로 변환 (각 시설마다)
distances_km = distances[0] * 6371  

# 찾은 3개 시설 정보 출력
for i, (idx, dist) in enumerate(zip(indices[0], distances_km), start=1):
    facility = df.iloc[idx]
    print(f"{i}번째 가까운 시설: {facility['시설명']}")
    print(f"주소: {facility['주소']}")
    print(f"거리: {dist:.2f} km\n")



1번째 가까운 시설: 서구재활방문요양센터
주소: 인천광역시 서구 탁옥로73번길 1
거리: 0.12 km

2번째 가까운 시설: 시온휴재가복지센터
주소: 인천광역시 서구 심곡로 53
거리: 0.15 km

3번째 가까운 시설: 돌핀투원복지용구
주소: 인천광역시 서구 심곡로 53
거리: 0.15 km



In [57]:
m = folium.Map(location=incheon_center, zoom_start=11)

In [59]:
# 사용자 위치 마커 추가
folium.Marker(location=user_location, popup='내 위치', icon=folium.Icon(color='red')).add_to(m)

# 가장 가까운 3개 시설 마커 강조 표시 (색상 변경 등)
for idx in indices[0]:
    facility = df.iloc[idx]
    folium.Marker(location=[facility['위도'], facility['경도']],
                  popup=facility['시설명'],
                  icon=folium.Icon(color='green')).add_to(m)

# 지도 다시 출력
m
